Note: this notebook is set up to run with the env.yml containing the name 'polaris_datasets'

## Background
The Drug Design Data Resource (D3R) aims to advance the technology of computer-aided drug discovery through the interchange of high quality protein-ligand datasets and workflows, and by holding community-wide, blinded prediction challenges. The D3R project is based at the University of California San Diego (UCSD), where it is co-directed by Drs. Rommie Amaro and Michael Gilson. An additional D3R component, focused on determining, validating and archiving protein-ligand co-crystal structures, is hosted at Rutgers the State University of New Jersey and led by Dr. Stephen K. Burley, who is Director of the RCSB Protein Data Bank.

## Assay Information
The cathepsins constitute an 11-member family of proteases involved in protein degradation. Cathepsin S is highly expressed in antigen-presenting cells, where it degrades major histocompatibility complex class II (MHC II)-associated invariant chain. CatS is a candidate target for regulating immune hyper-responsiveness, as the inhibition of CatS may limit antigen presentation. This data set comprises a follow-on challenge to GC3, consisting of non-peptidic, non-covalent, small molecule inhibitors across a three order of magnitude range (nM to μM) of IC50s for CatS. Specifically, we provide 459 CatS inhibitors for affinity prediction. This dataset was kindly donated by Janssen. Please note the affinity values from this set were measured against a C25S CatS mutant.
![GC3](https://drugdesigndata.org/upload/community-components/CatS.png)

Representative crystal structures of CatS.

## Description of readout:
- **AFFINITY**: Affinity of molecules for a C25S Cathepsin S mutant. 

## Data resource

**Raw data**: https://drugdesigndata.org/about/datasets/2028

## Curation reproducibility
The curation process in this notebook can be reproduced by command line:

```shell
auroris curate org-Biogen/d3r_cathepsin_c25s/curation_config.json org-Biogen/d3r_cathepsin_c25s
```

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pathlib

import pandas as pd
import datamol as dm

root = pathlib.Path("__file__").absolute().parents[2]
# set to recipe root directory
os.chdir(root)
sys.path.insert(0, str(root))

In [2]:
org = "polaris"
data_name = "d3r_cathepsin_c25s"
dirname = dm.fs.join(root, f"org-{org}", data_name)
gcp_root = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}"

/Users/lu.zhu/miniconda3/envs/po_dataset/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset was downloaded directly from [D3R](https://drugdesigndata.org/about/datasets) on 2024-03-22 by following the link under Name. We saved a version of the raw data as a parquet file in a Google Cloud bucket.

In [3]:
# Load the data
source_data_path = f"{gcp_root}/data/raw/D3R_CatS_GC4_raw.parquet"
data = pd.read_parquet(source_data_path)

In [4]:
# Keep only the SMILES, ID and outcome rows
columns_to_keep = ["Cmpd_ID", "SMILES", "Affinity"]
data = data[columns_to_keep].copy()
# Rename all columns to uppercase
for col in data.columns:
    data.rename(columns={col: col.upper()}, inplace=True)

In [5]:
data.describe(include="all")

,CMPD_ID,SMILES,AFFINITY
count,459,459,459.000000
unique,459,459,NaN
top,CatS_1,CC(C)N1CCN(CC1)C(=O)CSc2cc(ccc2C(F)(F)F)c3c4c(...,NaN
freq,1,1,NaN
mean,NaN,NaN,0.469236
std,NaN,NaN,0.943299
min,NaN,NaN,0.006500
25%,NaN,NaN,0.070000
50%,NaN,NaN,0.180000
75%,NaN,NaN,0.480000


### Perform data curation with `auroris.curation` module
The curation process includes:
- assign unique identifier to molecules
- detect the stereochemistry information of molecules.
- inspect the potential outliers of bioactivity values
- merge rows of replicated molecules
- detect isomers which show the activity shifts

Check out the curation module in [Auroris](https://github.com/polaris-hub/auroris). 

In [6]:
# Define data column names
data_cols = ["AFFINITY"]
mol_col = "SMILES"

In [7]:
# import key curation components from auroris
from auroris.curation import Curator
from auroris.curation.actions import (
    MoleculeCuration,
    OutlierDetection,
    Deduplication,
    StereoIsomerACDetection,
    ContinuousDistributionVisualization,
)

# Define the curation workflow
curator = Curator(
    data_path=source_data_path,
    steps=[
        MoleculeCuration(input_column=mol_col, y_cols=data_cols),
        ContinuousDistributionVisualization(y_cols=data_cols),
        OutlierDetection(
            method="zscore", columns=data_cols, threshold=3, use_modified_zscore=True
        ),
        StereoIsomerACDetection(y_cols=data_cols, threshold=3),
    ],
    parallelized_kwargs={"n_jobs": -1},
)

curator.to_json(f"{dirname}/inspection_config.json")

In [8]:
# Run the curation step defined as above
data_inspection, report = curator(data)

2024-07-10 01:59:32.630 | INFO     | auroris.curation._curator:transform:106 - Performing step: mol_curation
2024-07-10 01:59:48.766 | INFO     | auroris.curation._curator:transform:106 - Performing step: distribution
2024-07-10 01:59:48.808 | INFO     | auroris.curation._curator:transform:106 - Performing step: outlier_detection
2024-07-10 01:59:48.856 | INFO     | auroris.curation._curator:transform:106 - Performing step: ac_stereoisomer


In [9]:
#  get the curation logger
from auroris.report.broadcaster import LoggerBroadcaster

broadcaster = LoggerBroadcaster(report)
broadcaster.broadcast()

===== Curation Report =====
Time: 2024-07-10 01:59:32
Version: dev
===== mol_curation =====
[LOG]: New column added: MOL_smiles
[LOG]: New column added: MOL_molhash_id
[LOG]: New column added: MOL_molhash_id_no_stereo
[LOG]: New column added: MOL_num_stereoisomers
[LOG]: New column added: MOL_num_undefined_stereoisomers
[LOG]: New column added: MOL_num_defined_stereo_center
[LOG]: New column added: MOL_num_undefined_stereo_center
[LOG]: New column added: MOL_num_stereo_center
[LOG]: New column added: MOL_undefined_E_D
[LOG]: New column added: MOL_undefined_E/Z
[LOG]: Default `ecfp` fingerprint is used to visualize the chemical space.
[LOG]: Molecules with undefined stereocenter detected: 0.
[IMG]: Dimensions 1200 x 600
===== distribution =====
[IMG]: Dimensions 1200 x 600
===== outlier_detection =====
[LOG]: New column added: OUTLIER_AFFINITY
[LOG]: Found 9 potential outliers with respect to the AFFINITY column for review.
[IMG]: Dimensions 1200 x 600
===== ac_stereoisomer =====
[LOG]:

In [10]:
# Generate an HTML report with embedded visualizations showcasing the data analysis.
from utils.auroris_utils import HTMLBroadcaster

# export report to local directory
broadcaster = HTMLBroadcaster(report, f"{dirname}/inspection_report")
report_path = broadcaster.broadcast()

In [11]:
# check the curated data
data_inspection.describe(include="all")

,CMPD_ID,SMILES,AFFINITY,MOL_smiles,MOL_molhash_id,MOL_molhash_id_no_stereo,MOL_num_stereoisomers,MOL_num_undefined_stereoisomers,MOL_num_defined_stereo_center,MOL_num_undefined_stereo_center,MOL_num_stereo_center,MOL_undefined_E_D,MOL_undefined_E/Z,OUTLIER_AFFINITY,AC_AFFINITY
count,459,459,459.000000,459,459,459,459.000000,459.0,459.000000,459.0,459.000000,459,459.0,459,459
unique,459,459,NaN,459,459,449,NaN,NaN,NaN,NaN,NaN,1,1.0,2,2
top,CatS_1,CC(C)N1CCN(CC1)C(=O)CSc2cc(ccc2C(F)(F)F)c3c4c(...,NaN,CC(C)N1CCN(C(=O)CSc2cc(-c3nn(CCCN4CCC(N5CCCC5=...,381c727a5862b52f7ec5b14963ce711a0123c56c,289b4b0bc45a9fafdc6b082e0cb957c0a0fd249e,NaN,NaN,NaN,NaN,NaN,False,0.0,False,False
freq,1,1,NaN,1,1,2,NaN,NaN,NaN,NaN,NaN,459,459.0,450,457
mean,NaN,NaN,0.469236,NaN,NaN,NaN,1.738562,1.0,0.631808,0.0,0.631808,NaN,NaN,NaN,NaN
std,NaN,NaN,0.943299,NaN,NaN,NaN,0.956051,0.0,0.662099,0.0,0.662099,NaN,NaN,NaN,NaN
min,NaN,NaN,0.006500,NaN,NaN,NaN,1.000000,1.0,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,0.070000,NaN,NaN,NaN,1.000000,1.0,0.000000,0.0,0.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,0.180000,NaN,NaN,NaN,2.000000,1.0,1.000000,0.0,1.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,0.480000,NaN,NaN,NaN,2.000000,1.0,1.000000,0.0,1.000000,NaN,NaN,NaN,NaN


## Signals or outliers
This process utilized `zscore` as the default method, but one can adjust the outlier detection method by defining parameters within the `method`. \
For more information and details on this, please refer to `auroris.curation.actions.OutlierDetection`.

During the curation process, several potential outliers were flagged across multiple endpoints. These outliers have been marked and included in the curated output. 


![Affinity](inspection_report/images/2-Outlier_detection_-_AFFINITY.png)

It's worth noting that the flagged outliers (highlighted in red), which are located at the extremes of the data distributions, are still in the value range of readout `AFFINITY` measurement and are likely to be false positive outliers. Therefore, they should be examined closely.

## Chemical space coverage of the dataset

![chemical space chem_all](inspection_report/images/0-Distribution_in_Chemical_Space_-_ECFP.png)

The above plot shows that the molecules in dataset are distributed in 5 groups in the chemical space.

### Examples of stereoisomers having activity shift

![AC_stereo](inspection_report/images/3-Detection_of_activity_shifts_among_stereoisomers.png)

The `AFFINITY` values indicate that the binding affinity of the isomer on the left is much more higher than the isomer on the right. 

## Rerun data curation and export curated data for downstream tasks

In [12]:
# Define the final curation workflow
curator = Curator(
    source_data=source_data_path,
    steps=[
        MoleculeCuration(input_column=mol_col, y_cols=data_cols),
        ContinuousDistributionVisualization(y_cols=data_cols),
        Deduplication(
            deduplicate_on=mol_col, y_cols=data_cols
        ),  # remove the replicated molecules
        OutlierDetection(
            method="zscore", columns=data_cols, threshold=3, use_modified_zscore=True
        ),
        StereoIsomerACDetection(y_cols=data_cols, threshold=3),
    ],
    parallelized_kwargs={"n_jobs": -1},
)

In [13]:
# The final curation configuration is exported for reproducibility
path = f"{gcp_root}/data/curation/curation_config.json"
curator.to_json(path)

In [14]:
# Run the curation step defined as above
data_curated, report = curator(data)

2024-07-10 01:59:50.824 | INFO     | auroris.curation._curator:transform:106 - Performing step: mol_curation
2024-07-10 01:59:53.013 | INFO     | auroris.curation._curator:transform:106 - Performing step: distribution
2024-07-10 01:59:53.057 | INFO     | auroris.curation._curator:transform:106 - Performing step: deduplicate
2024-07-10 01:59:53.265 | INFO     | auroris.curation._curator:transform:106 - Performing step: outlier_detection
2024-07-10 01:59:53.317 | INFO     | auroris.curation._curator:transform:106 - Performing step: ac_stereoisomer


In [15]:
broadcaster = LoggerBroadcaster(report)
broadcaster.broadcast()

===== Curation Report =====
Time: 2024-07-10 01:59:50
Version: dev
===== mol_curation =====
[LOG]: New column added: MOL_smiles
[LOG]: New column added: MOL_molhash_id
[LOG]: New column added: MOL_molhash_id_no_stereo
[LOG]: New column added: MOL_num_stereoisomers
[LOG]: New column added: MOL_num_undefined_stereoisomers
[LOG]: New column added: MOL_num_defined_stereo_center
[LOG]: New column added: MOL_num_undefined_stereo_center
[LOG]: New column added: MOL_num_stereo_center
[LOG]: New column added: MOL_undefined_E_D
[LOG]: New column added: MOL_undefined_E/Z
[LOG]: Default `ecfp` fingerprint is used to visualize the chemical space.
[LOG]: Molecules with undefined stereocenter detected: 0.
[IMG]: Dimensions 1200 x 600
===== distribution =====
[IMG]: Dimensions 1200 x 600
===== deduplicate =====
[LOG]: Deduplication merged and removed 0 duplicated molecules from dataset
===== outlier_detection =====
[LOG]: New column added: OUTLIER_AFFINITY
[LOG]: Found 9 potential outliers with respec

In [16]:
# Export report to polaris public directory on GCP
# The report is ready to reviewed in the HTML file.
broadcaster = HTMLBroadcaster(
    report, f"{gcp_root}/data/curation/report", embed_images=True
)
broadcaster.broadcast()

'gs://polaris-public/polaris-recipes/org-polaris/d3r_cathepsin_c25s/data/curation/report/index.html'

## Export the final curated data

In [17]:
fout = f"{gcp_root}/data/curation/{data_name}_curated.csv"
data_curated.reset_index(drop=True).to_csv(fout, index=False)